## Generate Random Coordinates

In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from scipy.stats import linregress
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime
# Import the API key.
from config import weather_api_key

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
coordinates = list(lat_lngs)

## Find Unique Cities Based on Coordinates

In [3]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

In [ ]:
# Base URL for Weather Map API Call.
url = f"http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID={weather_api_key}"

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | mataura
Processing Record 2 of Set 1 | fairbanks
Processing Record 3 of Set 1 | rikitea
Processing Record 4 of Set 1 | grand-santi
Processing Record 5 of Set 1 | jamestown
Processing Record 6 of Set 1 | mar del plata
Processing Record 7 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 8 of Set 1 | sorvag
City not found. Skipping...
Processing Record 9 of Set 1 | pacific grove
Processing Record 10 of Set 1 | mangrol
Processing Record 11 of Set 1 | avarua
Processing Record 12 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 13 of Set 1 | severo-kurilsk
Processing Record 14 of Set 1 | constitucion
Processing Record 15 of Set 1 | atuona
Processing Record 16 of Set 1 | sao filipe
Processing Record 17 of Set 1 | mbala
Processing Record 18 of Set 1 | cabo san lucas
Processing Record 19 of Set 1 | saleaula
City not found. Skipping...
Processing Record 20 of 

Processing Record 38 of Set 4 | mingguang
Processing Record 39 of Set 4 | katsuura
Processing Record 40 of Set 4 | coleraine
Processing Record 41 of Set 4 | mareeba
Processing Record 42 of Set 4 | maceio
Processing Record 43 of Set 4 | butaritari
Processing Record 44 of Set 4 | verkhnyaya toyma
Processing Record 45 of Set 4 | mumford
Processing Record 46 of Set 4 | kerteh
City not found. Skipping...
Processing Record 47 of Set 4 | pangnirtung
Processing Record 48 of Set 4 | awjilah
Processing Record 49 of Set 4 | darnah
Processing Record 50 of Set 4 | bauska


## Add Info to Dataframe and Convert to CSV

In [ ]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df = city_data_df[["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]]
city_data_df.head(10)

In [ ]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")